## This Notebook - Goals - FOR EDINA 

**What?:** <br>
- Introduction/tutorial to <code>folium</code>, an interactive library for plotting data on leaflet maps
- Visualization of current Covid-19 data

**Who?:** <br>
- Academics in geosciences
- Geophysical Data Science course
- Users interested in geospatial data analysis
 
**Why?:** <br>
- Tutorial/guide for academics and students on how to folium and how to process data

**Noteable features to exploit:** <br>
- Use of pre-installed libraries 

**How?:** <br>
- Step-by-step data processing using real-time global Covid-19 data
- Clear visualisations - concise explanations
- Effective use of core libraries
<hr>

# Folium 
<code>folium</code> makes it easy to visualize data that’s been manipulated in Python on an interactive leaflet map. It enables both the binding of data to a map for choropleth visualizations as well as passing rich vector/raster/HTML visualizations as markers on the map. The library has a number of built-in tilesets from <code>OpenStreetMap</code>, but <code>Stamen</code> and <code>Mapbox</code> and supports custom tilesets with Mapbox or Cloudmade API keys. It supports both Image, Video, GeoJSON and TopoJSON overlays. It also has several plugins that enable interactive tools for the maps.

This notebook is split into two sections to illustrate the two main uses of folium. The first section showcases how to plot markers, polylines and other vector/raster/HTML visualizations on a basemap. The second half of the notebook is a tutorial on creating a global choropleth map of the confirmed cases of Covid-19 for each country. There is a step-by-step guide to access, process and clean up the data before plotting it. The tutorial runs the user through the optimal way of visualizing the data, starting with a linear segmented colormap, then a logarithmic colormap and a logarithmic colormap with hover tools showing the absolute values for each country.

**Notebook contents:**
- Importing the necessary libraries
- <a href='#marker'>Adding vector/raster/HTML visualizations on a map</a>
- <a href='#pre'>Pre-processing of open source Covid-19 data</a>
- Choropleth maps using Covid-19 data
    - <a href='#lin'>Linear colormap</a>
    - <a href='#log'>Logarithmic colormap</a>
    - <a href='#hov'>Logarithmic colormap with hover tools showing absolute values</a>

In [1]:
# Import general libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import requests
import json
from IPython.display import display

# Import folium package and modules
import folium
from folium import plugins

# Import widgets library
import ipywidgets as widgets
from ipywidgets import interactive

<a id='marker'></a>
## Adding vector/raster/HTML visualizations on a map
You can easily create a basemap using <code>folium.Map()</code>. The default tiles are set to <code>OpenStreetMap</code>, but <code>Stamen Terrain</code>, <code>Stamen Toner</code>, <code>Mapbox Bright</code>, and <code>Mapbox Control Room</code>, and many other tiles are built in as well. 

There are numerous marker types, starting with a simple Leaflet style location marker with a popup and tooltip HTML which can be created by specifying the coordinates within <code>folium.Marker()</code> and added to the map by <code>.add_to(map)</code>. For the full list of marker types, follow this [link](https://python-visualization.github.io/folium/modules.html).

There are also several plugin features you can add to the map to make it more interactive such as <code>plugins.Fullscreen()</code>. You can find the full list [here](https://python-visualization.github.io/folium/plugins.html).

In [2]:
# Create the basemap
loc = [55.9533, -3.1883] # Coordinates of Edinburgh
m1 = folium.Map(location=loc, # Map centred at Edinburgh
                tiles='OpenStreetMap', 
                zoom_start=13) # Zoomed in to show central Edinburgh

# Add pop-up showing the coordinates when clicking anywhere on the map
m1.add_child(folium.LatLngPopup())

# Add a marker at Argyle House (Noteable's offices)
tooltip = 'Click me!'
locationName = 'Work' # Text for pop-up
workloc = [55.9465, -3.2007] # Coordinates of Argyle House
folium.Marker(workloc, popup=locationName, tooltip=tooltip).add_to(m1) # Add the marker to basemap

# Add a red circle to the map
folium.Circle(radius=400, # Radius of circle
              location=workloc, # Coordinates of centre of circle
              popup='Possible places for lunch', # Text for pop-up
              color='crimson',
              fill=False,
              tooltip=tooltip).add_to(m1) # Adding circle to basemap

# Add a marker at your location
locationName2 = 'Home' # Text for pop-up
homeloc = [55.9399, -3.1773] # Coordinates of your location
folium.Marker(homeloc, popup=locationName2, tooltip=tooltip).add_to(m1) ## Add the marker to basemap

# Coordinates of polyline between the 2 markers
path = [(55.9465, -3.2007), (55.9461, -3.2010), (55.9452, -3.1996), (55.9449, -3.1991), 
        (55.9449, -3.1952), (55.9453, -3.1912), (55.9426, -3.1914), (55.9423, -3.1904), 
        (55.9407, -3.1854), (55.9405, -3.1850), (55.9403, -3.1840), (55.9409, -3.1809), 
        (55.9399, -3.1802), (55.9405, -3.1780), (55.9399, -3.1773)]

# Add antpath plugin to show the path between the 2 markers
walk = 'Walk to work'
plugins.AntPath(locations=path, smooth_factor=5, popup=walk, tooltip=tooltip).add_to(m1)

# Add plugin to make map full screen
plugins.Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen').add_to(m1)

# Add plugin to measure distances and areas
plugins.MeasureControl().add_to(m1)

# Uncomment if you comment out the section below ###  ###
#display(m1) # Display map

### The following part is to give an insight into the different tile options - not needed to plot the map ###

# Define function to plot map with the selected tile
def tiles(tilemap):
    folium.raster_layers.TileLayer(tiles=tilemap, overlay=True).add_to(m1) # Change tiles to selected one
    display(m1) # Display map

# Create Dropdown widget for tile selection
tilemap=widgets.Dropdown(options=['OpenStreetMap', 'Stamen Terrain', 'Stamen Toner', 
                                  'Stamen Watercolor', 'CartoDB positron', 'CartoDB dark_matter'],
                         value='OpenStreetMap',
                         description='Choose base layer:',
                         continuous_update=True,
                         disabled=False,
                         style={'description_width': 'initial'})

# Create interative output
out = widgets.interactive_output(tiles, {'tilemap':tilemap})

# Display interactive widgets in UI
display(tilemap, out)

Dropdown(description='Choose base layer:', options=('OpenStreetMap', 'Stamen Terrain', 'Stamen Toner', 'Stamen…

Output()

<a id='pre'></a>
## Pre-processing Covid-19 data

### Getting the data
Open source global data on the number of confirmed Covid-19 cases per country can be accessed through the following github repo: https://github.com/CSSEGISandData/COVID-19.git <br>
The entire repository can be cloned into your Noteable home directory, although it might take a few tries and a few minutes for the repo to be succesfully cloned as it is very large.

In [3]:
# Clone github repository of open source Covid-19 data - sometimes takes a few tries and takes a few minutes as it is quite a large repo
!git clone https://github.com/CSSEGISandData/COVID-19.git

Cloning into 'COVID-19'...
remote: Enumerating objects: 46552, done.
remote: Total 46552 (delta 0), reused 0 (delta 0), pack-reused 46552
Receiving objects: 100% (46552/46552), 360.42 MiB | 3.72 MiB/s, done.
Resolving deltas: 100% (29006/29006), done.
Checking out files: 100% (564/564), done.


### Cleaning up the data
After successfully cloning the repo into your home directory, the data used can be found in <code>COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv</code>. If you have a look inside the csv file, you can see that most of the data is available per country. However, there are some countries which have data split into states or provinces. These had to be summed up to give the overall number of cases within that country.

Another issue is that the data doesn't have any vector data associated with the countries, i.e. no polygons are defined for the choropleth map to plot. So the vecor geometry for the countries had to be obtained from raw github data available: https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json. As some of the country names in the Covid-19 csv data and the vector geometry json file differ, they need to be changed to match. Greenland, for example, was a special case because it is an autonomous territory within the Kingdom of Denmark so it was listed under Greenland, Denmark in the csv file, but it had its own polygon geometry within the json file. Therefore the country name had to be changed to Greenland to match the json file.

After having fixed all the duplicate countries, several columns in the csv file become redundant (Latitude, Longitude and Province/State columns) so these can be dropped to make the data look cleaner. All '0' values are changed to 'NaN' as it would later become an issue when the data is plotted on a logarithmic scale (log0 is undefined).

To be able to plot the data with a hover tooltip, the Covid-19 data needs to be added to the the json file. So first, the csv file needs to be converted to json as well and then the right keys and values need to be selected from both json files.

What needs to be done:
- need vector geometry from additional json file
- match country names in csv and json files
- sum number of cases of countries split into states or provinces into one country wide statistic
- fix Greenland's country name
- drop duplicate countries - keep country-wide data
- change all '0' values to 'NaN'
- drop Latitude, Longitude and State/Province columns
- convert csv file into json and add Covid-19 data to vector geometry json file

In [12]:
# Load timeseries data of daily cumulative confirmed infections per country
total_cases_g = pd.read_csv('COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', na_values=0, delimiter=',')

# Match country names in dataframe(eg. 'US') to country names in json file (eg. 'United States of America')
total_cases_g.replace('US', 'United States of America', inplace = True)
total_cases_g.replace('Tanzania', "United Republic of Tanzania", inplace = True)
total_cases_g.replace('Congo (Brazzaville)', "Republic of the Congo", inplace = True)
total_cases_g.replace('Congo (Kinshasa)', "Democratic Republic of the Congo", inplace = True)
total_cases_g.replace('Czechia', "Czech Republic", inplace = True)
total_cases_g.replace('Korea, South', "South Korea", inplace = True)
total_cases_g.replace('Taiwan*', "Taiwan", inplace = True)
total_cases_g.replace('Serbia', "Republic of Serbia", inplace = True)
total_cases_g.replace('North Macedonia', "Macedonia", inplace = True)
total_cases_g.replace('Guinea-Bissau', "Guinea Bissau", inplace = True)
total_cases_g.replace("Cote d'Ivoire", "Ivory Coast", inplace = True)

# Drop columns of coordinates - using geojson file to plot the polygons of the countries so this is not necessary
total_cases_g = total_cases_g.drop(columns=['Lat', 'Long'])

In [13]:
# Show duplicated countries
total_cases_g[total_cases_g.duplicated(subset='Country/Region') == True]

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20
9,New South Wales,Australia,NaN,NaN,NaN,NaN,3.0,4.0,4.0,4.0,...,3851.0,3861.0,3875.0,3897.0,3915.0,3927.0,3936.0,3945.0,3950.0,3957.0
10,Northern Territory,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0
11,Queensland,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1088.0,1088.0,1089.0,1089.0,1089.0,1089.0,1091.0,1091.0,1091.0,1091.0
12,South Australia,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,459.0,459.0,459.0,459.0,459.0,459.0,460.0,460.0,460.0,461.0
13,Tasmania,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,229.0,229.0,229.0,229.0,230.0,230.0,230.0,230.0,230.0,230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,Gibraltar,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,190.0,197.0,201.0,201.0,202.0,203.0,205.0,206.0,209.0,215.0
253,Isle of Man,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,336.0,336.0,336.0,336.0,336.0,336.0,336.0,336.0,336.0,336.0
254,Montserrat,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0
255,Turks and Caicos Islands,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,170.0,170.0,216.0,216.0,216.0,241.0,258.0,274.0,298.0,298.0


In [14]:
# Combine states/provinces of the duplicated countries to one country
countries = ['Australia', 'Canada', 'United Kingdom', 'France', 'Netherlands', 'China'] # Duplicated countries
for i in countries:
    series = total_cases_g[total_cases_g['Country/Region'] == i].agg('sum') # Sum up all content of all columns
    series['Country/Region'] = i # Rename Country name
    series['Province/State'] = 'NaN' # Rename Province/State name to NaN
    df = series.to_frame().transpose() # Turn series into a dataframe and switch rows and columns 
    total_cases_g = pd.concat([total_cases_g, df], ignore_index=True, sort = True) # Combine original datframe and new dataframe of fixed duplicates

# Fix naming issue with Greenland - can't do what we did with the others as it exists in geojson file
greenland = total_cases_g[total_cases_g['Province/State'] == 'Greenland'] # Create dataframe with only greenland in it
greenland['Country/Region'] = 'Greenland' # Rename country
greenland['Province/State'] = 'NaN' # Rename Province/State
total_cases_g = pd.concat([total_cases_g, greenland], ignore_index=True) # Combine original dataframe and greenland dataframe

# Replace 0 with NaN to make sure np.log used on values later on is defined
total_cases_g.replace(0, np.nan, inplace=True)

# Drop duplicate countries and keep the last (combined data) entry
total_cases_g.drop_duplicates(subset='Country/Region', keep='last', inplace=True)

# Delete 'Province/State' columns as it is not needed anymore
total_cases_g.drop(columns=['Province/State'], inplace=True)

# Sort dataframe alphabetically by country
total_cases_g.sort_values('Country/Region', inplace = True)

# Show final edited dataframe
total_cases_g

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,8/16/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,Country/Region
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37596.0,36710.0,36747.0,36782.0,36829.0,36896.0,37015.0,37054.0,37054.0,Afghanistan
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7380.0,5519.0,5620.0,5750.0,5889.0,6016.0,6151.0,6275.0,6411.0,Albania
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38583.0,31465.0,31972.0,32504.0,33055.0,33626.0,34155.0,34693.0,35160.0,Algeria
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,989.0,925.0,937.0,939.0,939.0,944.0,955.0,955.0,955.0,Andorra
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1906.0,1199.0,1280.0,1344.0,1395.0,1483.0,1538.0,1572.0,1672.0,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16534.0,12297.0,12541.0,12770.0,13065.0,13398.0,13722.0,13928.0,14208.0,West Bank and Gaza
262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,Western Sahara
263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1869.0,1734.0,1734.0,1760.0,1763.0,1768.0,1796.0,1797.0,1804.0,Yemen
264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9343.0,6347.0,6580.0,6793.0,7022.0,7164.0,7486.0,7903.0,8085.0,Zambia


In [15]:
#Setting up the world countries data URL
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

# Loading it into the notebook as a json file
res = requests.get(country_shapes)
data = res.json()

In [8]:
# Set index of covid-dataframe to the country
total_cases_ind = total_cases_g.set_index('Country/Region')

# Turn dataframe into a json file with a structure of  {column -> {index -> value}}
result = total_cases_ind.to_json(orient='columns')
parsed = json.loads(result)

# Select any date to show all the 'country : data' pairs
parsed["7/22/20"]

{'Afghanistan': 35727.0,
 'Albania': 4358.0,
 'Algeria': 24872.0,
 'Andorra': 889.0,
 'Angola': 812.0,
 'Antigua and Barbuda': 76.0,
 'Argentina': 141900.0,
 'Armenia': 35693.0,
 'Australia': 13302.0,
 'Austria': 19929.0,
 'Azerbaijan': 28633.0,
 'Bahamas': 219.0,
 'Bahrain': 37637.0,
 'Bangladesh': 213254.0,
 'Barbados': 106.0,
 'Belarus': 66521.0,
 'Belgium': 64627.0,
 'Belize': 43.0,
 'Benin': 1690.0,
 'Bhutan': 92.0,
 'Bolivia': 64135.0,
 'Bosnia and Herzegovina': 9115.0,
 'Botswana': 522.0,
 'Brazil': 2227514.0,
 'Brunei': 141.0,
 'Bulgaria': 9584.0,
 'Burkina Faso': 1066.0,
 'Burma': 343.0,
 'Burundi': 328.0,
 'Cabo Verde': 2154.0,
 'Cambodia': 198.0,
 'Cameroon': 16522.0,
 'Canada': 113790.0,
 'Central African Republic': 4574.0,
 'Chad': 889.0,
 'Chile': 336402.0,
 'China': 85906.0,
 'Colombia': 218428.0,
 'Comoros': 337.0,
 'Costa Rica': 12361.0,
 'Croatia': 4530.0,
 'Cuba': 2462.0,
 'Cyprus': 1040.0,
 'Czech Republic': 14570.0,
 'Democratic Republic of the Congo': 8626.0,
 'De

<a id='lin'></a>
## Linear colormap
Once all the data is processed, actually plotting it is quite straightforward using <code>folium.Choropleth()</code>. The arguments needed within the function are <code>geo_data</code> - which is the json file - and <code>data</code> - which is the cleaned csv file - with the data columns specified in <code>columns</code>. If no bins are specified for the colormap scale, it automatically plots it on a linear scale with equal bin widths.

This however might not be the most efficient way to plot the data. Because of a few very large values, the range of the colormap scale is so wide that the shading on the map doesn't appear to be very informative. So there might be other ways to visualize the data in a better way.

In [9]:
# Define function to enable using Date Picker widget
def mapping(date):
    # Change format of date picked by widget to match the column names of the dataframe
    date1 = date.strftime("%-m/%-d/%y")
    # Create basemap layer
    m = folium.Map()
    # Create choropleth layer
    folium.Choropleth(geo_data=country_shapes, # Geometry for countries
                      name='choropleth COVID-19', # Name of layer
                      data=total_cases_g, columns=['Country/Region', date1], # Covid-19 data per country
                      key_on='feature.properties.name', # Matching Covid-19 data to the geometry by the country name
                      legend_name='Total confirmed cases of Covid-19 until '+ str(date1), # Colorbar label
                      highlight=True, # Highlighting country polygons when hovering over it
                      fill_color='YlOrRd', # Colormap
                      nan_fill_color='white', # Color for NaN values
                     ).add_to(m) # Add choropleth layer to basemap
    # Add control panel to choose which layer to show on the map
    folium.LayerControl().add_to(m) 
    # Show map
    display(m) 
    return date1

# Create Date Picker widget
date = widgets.DatePicker(description='Pick a Date', value = datetime.date(2020, 7, 22), disabled=False)

# Make widget interactive by connecting it to the function 
interactive(mapping, date=date)

interactive(children=(DatePicker(value=datetime.date(2020, 7, 22), description='Pick a Date'), Output()), _dom…

<a id='log'></a>
## Logarithmic colormap
If you have a look at the data then you can see that a few countries - usually ones with large populations -  have huge numbers, but most are relatively small in comparison. One way around this is to plot the data on a logarithmic scale. This makes the choropleth map look more representative as you can see the differences in the number of cases between smaller countries with less population. This, however, means the ticks on the colorbar go from 1-15, which isn't very informative about the absolute values for the countries. So the next step would be to add additional hover tools which shows the absolute number of cases for each country when you hover over the country.

*Note: An interesting exercise would be to create a colormap where outlying values are coloured differently so that the colormap range is more representative of the number of cases most countries have. Another interesting way to visualize the data would be to obtain data on the population and visualize the percentage of the number of confirmed Covid-19 cases with respect to the country's population. A third option would be to process the data further and obtain the number of new daily confirmed Covid-19 cases instead of plotting the cumulative value. All of these options should lessen the effect of the outliers and influence of the country's population.*

In [10]:
# Define function to enable using Date Picker widget
def mapping(date):
    # Change format of date picked by widget to match the column names of the dataframe
    date1 = date.strftime("%-m/%-d/%y")
    # Create basemap layer
    m = folium.Map()
    # Change absolute values into a logarithmic scale
    total_cases_g['day'] = np.log(total_cases_g[date1])
    # Create choropleth layer
    folium.Choropleth(geo_data=country_shapes, # Geometry for countries
                      name='choropleth COVID-19', # Name of layer
                      data=total_cases_g, columns=['Country/Region', 'day'], # Covid-19 data per country
                      key_on='feature.properties.name', # Matching Covid-19 data to the geometry by the country name
                      legend_name='Log of total confirmed cases of Covid-19 until '+ str(date1), # Colorbar label
                      highlight=True, # Highlighting country polygons when hovering over it
                      fill_color='YlOrRd', # Colormap
                      nan_fill_color='white', # Color for NaN values
                     ).add_to(m)# Add choropleth layer to basemap
    # Add control panel to choose which layer to show on the map
    folium.LayerControl().add_to(m)
    # Show map
    display(m)
    return date1

# Create Date Picker widget
date = widgets.DatePicker(description='Pick a Date', value = datetime.date(2020, 7, 22), disabled=False)

# Make widget interactive by connecting it to the function 
interactive(mapping, date=date)

interactive(children=(DatePicker(value=datetime.date(2020, 7, 22), description='Pick a Date'), Output()), _dom…

<a id='hov'></a>
## Logarithmic colorbar with hover tools
As you have seen above, the color scheme looks reasonable but obtaining the absolute values require some calculation as the colorbar is given in a logarithmic scale. To avoid unnecessary calculations and make the plot easily interpreted, a hover tool can be added to display <code>key : value</code> pairs from the geometry json file. Therefore, the data containing the number of confirmed Covid-19 cases has to be added to the geometry json file - this also requires the conversion of the csv file to a json file.

This way, the colors are representative and the absolute values are also easily accessed.

In [11]:
# Define function to enable using Date Picker widget
def mapping(date):
    # Change format of date picked by widget to match the column names of the dataframe
    date1 = date.strftime("%-m/%-d/%y")
    
    # Add Covid-19 data of chosen date to the right dictionary in the json file
    for key in parsed[date1].keys(): # For each country for that chosen date within the Covid data (converted to json)
        for i in range(0, 177): # For each country in the geometry json file
            if data['features'][i]['properties']['name'] == key: # If country name in the geometry json file equals country name in covid json file
                data['features'][i]['properties']['number of confirmed cases'] = parsed[date1].get(key, 'NO') # Add new key: value pair to dictionary with covid-19 data
                
    # Create basemap layer
    m = folium.Map()
    # Change absolute values into a logarithmic scale
    total_cases_g['day'] = np.log(total_cases_g[date1])
    # Create choropleth layer
    choropleth = folium.Choropleth(geo_data=data, # Geometry for countries
                                   name='choropleth COVID-19', # Name of layer
                                   data=total_cases_g, columns=['Country/Region', 'day'], # Covid-19 data per country
                                   key_on='feature.properties.name', # Matching Covid-19 data to the geometry by the country name
                                   legend_name='Log of total confirmed cases of Covid-19 until '+ str(date1), # Colorbar label
                                   highlight=True, # Highlighting country polygons when hovering over it
                                   fill_color='YlOrRd', # Colormap
                                   nan_fill_color='white', # Color for NaN values
                                  ).add_to(m) # Add choropleth layer to basemap
    # Add hover tool showing name of the country and number of confirmed Covid-19 cases until the chosen date
    choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['name', 'number of confirmed cases']))
    # Add control panel to choose which layer to show on the map
    folium.LayerControl().add_to(m)
    # Show map
    display(m)
    return date1

# Create Date Picker widget
date = widgets.DatePicker(description='Pick a Date', value = datetime.date(2020, 7, 22), disabled=False)

# Make widget interactive by connecting it to the function 
interactive(mapping, date=date)

interactive(children=(DatePicker(value=datetime.date(2020, 7, 22), description='Pick a Date'), Output()), _dom…